# IronHack Project 01. Module 01

https://github.com/potacho/ih_datamadpt0420_project_m1

#### Imports

In [1]:
import pandas as pd
import sqlalchemy
import requests
import bs4
from functools import reduce

### Acquisition

#### DataBase

In [2]:
def read_database(ddbb):
    """
    Given a database, return all tables within. 
    
    Parameters:
        ddbb: String. SQL database path
        
    Return:
        4 Pandas data frames, personal_df, country_df, career_df and poll_df
    """
    
    # Establish connection
    engine = sqlalchemy.create_engine(ddbb)
    
    # Fetch tables
    personal_df = pd.read_sql_table("personal_info", engine)
    country_df = pd.read_sql_table("country_info", engine) 
    career_df = pd.read_sql_table("career_info", engine) 
    poll_df = pd.read_sql_table("poll_info", engine)
    
    return personal_df, country_df, career_df, poll_df

In [3]:
personal_df, country_df, career_df, poll_df = read_database("sqlite:///../data/raw/raw_data_project_m1.db")
personal_df.head()

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65


#### API

In [4]:
def retrieve_job_title(job_code):
    """
    Given a uuid, return uuid job title.
    E.g.: 861a9b9151e11362eb3c77ca914172d0 -> Automatic Data Processing Planner
    
    Parameters:
        job_code: String. Unique job code.
    
    Return:
        Job title. String.
    """
    if job_code is not None: # if job_code is not None.
        try:
            r = requests.get(f"http://api.dataatwork.org/v1/jobs/{job_code}")
            title = r.json()["title"]
        except:
            title = "Not found"
        
        return title
    
    else:
        return None

In [5]:
def create_titles_df(df, column):
    """
    Given a pandas data frame and a column of the same df. 
    Return a pandas dataframe comprised by two columns; normalized job code and its corresponding title.
    
    Parameters:
        df: Pandas DataFrame.
        column. String. df column comprised by normalized job codes e.g.: c1fb1a01b78373ac2153c66fa08d16dc
    
    Returns:
        Pandas dataframe comprised by two columns; job_code, job_title
    """
    
    # Create list with unique values.
    job_titles = list(df[column].unique())
    
    # Delete None at the first element. 
    if job_titles[0] is None:
        job_titles.pop(0)
    
    # Retrieve job's title.
    job_titles_names = [[x, retrieve_job_title(x)] for x in job_titles]
    
    # Create and return data frame.
    df = pd.DataFrame(job_titles_names, columns=["job_code", "job_title"])
    return df

In [6]:
titles_df = create_titles_df(career_df, "normalized_job_code")
titles_df.head()

,job_code,job_title
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner
1,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator
2,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer
3,27af8700f5577cec835acee2cb90a2ff,Data Entry Specialist
4,c1b670eba9ccb65e7c99f7da116d5b9c,Database Architect


#### Eurostat

In [7]:
def fetch_country_codes(url):
    """
    Retrive country names and codes from EuroStat web page.
    
    Parameters:
        url: String. Eurostat web page to parse
    
    Returns:
        Pandas dataframe comprised by two columns; "country" and "code"
    """
    
    # Retrieve raw html
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content)
    tables = soup.find_all("table")
    
    # Parse raw html
    country_codes = []
    for table in tables:
        trs = table.find_all("tr")
        for tr in trs:
            tds = tr.find_all("td")
            for td in tds:
                txt = td.text.rstrip("\n").lstrip()
                if len(txt) > 0:
                    if txt == "(TW)":
                        country_codes.append(txt)
                        #country_codes.append("Canada")
                    else:
                        country_codes.append(txt)

    # Split countries and codes into two different lists. 
    countries = [x for x in country_codes if country_codes.index(x) % 2 == 0]
    codes = [x for x in country_codes if country_codes.index(x) % 2 > 0]
    
    # Adding Greece(GR) and Great Britain(GB) manually.
    countries = countries + ["Greece", "Great Britain"]
    codes = codes + ["GR", "GB"]

    # Take previously split lists and create a single df
    df = pd.DataFrame(zip(countries, codes), columns=["country", "code"])
    df.code = df.code.apply(lambda x: x.lstrip("(").rstrip(")"))
    return df

In [8]:
url = "https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes"
country_code_df = fetch_country_codes(url)
country_code_df.tail()

,country,code
69,Japan,JP
70,Singapore,SG
71,United States,US
72,Greece,GR
73,Great Britain,GB


### Wrangling

#### Personal, Country and Career DataFrames

In [9]:
def personal_df_enhancement(df):
    """
    Take personal_df data frame and clean its columns.
    
    Parameters:
        df: Pandas DataFrame. Original personal_df coming from read_database function.
        
    Returns:
        Enhanced personal_df Data frame.
    """
    
    # juvenile will be switched to '14_25' because it matches with the age column.
    df.age_group = df.age_group.apply(lambda x: x.replace("juvenile", "14_25").strip())

    # Yes and No in the dem_has_children column will be capitalized.
    df.dem_has_children = df.dem_has_children.apply(lambda x: x.capitalize().strip())

    # Male and Female in the gender column will be capitalized, Fem will become Female as well. 
    df.gender = df.gender.apply(lambda x: x.replace("Fem", "FEMALE"))
    df.gender = df.gender.apply(lambda x: x.capitalize().strip())

    # Remove ' years old' from the age column and compute the age of those who only show the birth year, i.e.: 1991
    df.age = df.age.apply(lambda x: x.replace("years old", "").strip())
    df.age = df.age.apply(lambda x: 2016 - int(x) if len(x) > 3 else x)

    # Setting specific data types for each column.
    df.age = df.age.astype("int")
    df.gender = df.gender.astype("category")
    df.dem_has_children = df.dem_has_children.astype("category")
    df.age_group = df.age_group.astype("category")
    
    return df

In [10]:
def country_df_enhancement(df, country_codes_df):
    """
    Enhance country_df data frame by cleaning its columns and merging it with the country codes data frame. 
    
    Parameters:
        df: Pandas DataFrame. Original country_df coming from read_database function.
        country_codes_df: Pandas DataFrame comming from the fetch_country_codes function. 
        
    Returns:
        Enhanced country_df Data frame.
    """

    # Capitalize all categories in the rural column
    df.rural = df.rural.apply(lambda x: x.capitalize().strip())
    
    # Remove possible trailling spaces and turn into all caps all country codes.
    df.country_code = df.country_code.apply(lambda x: x.upper().strip())
    
    # Merge country_df and country_code_df and drop Code column
    df = pd.merge(df, country_codes_df, how="left", left_on="country_code", right_on="code")
    df.drop(columns="code", inplace=True)
    
    # Setting specific data types for each column
    df.rural = df.rural.astype("category")
    
    return df

In [11]:
def career_df_enhancement(df, job_titles_df):
    """
    Enhance career_df data frame by cleaning its columns and merging it with the job titles data frame. 
    
    Parameters:
        df: Pandas DataFrame. Original career_df coming from read_database function.
        job_titles_df: Pandas DataFrame comming from the create_titles_df function. 
        
    Returns:
        Enhanced career_df Data frame.
    """
    
    # Fills None in dem_education_level with "no"
    df.dem_education_level.fillna("no", inplace=True)
    
    # Capitalize column's values
    df[["dem_education_level", "dem_full_time_job"]] = df[["dem_education_level", "dem_full_time_job"]].apply(lambda x :x.str.capitalize())

    # Merge with titles_df
    df = pd.merge(df, titles_df, how="left", left_on="normalized_job_code", right_on="job_code")
    
    # Dropping job code columns
    df.drop(columns=["normalized_job_code", "job_code"], inplace=True)
    
    # Filling NaN in job title column with "unemployed"
    df.job_title.fillna("Unemployed", inplace=True)
    
    # Setting specific data types for each column
    df.dem_education_level = df.dem_education_level.astype("category")
    df.dem_full_time_job = df.dem_full_time_job.astype("category")
    
    return df

In [12]:
def merge_it(list_of_dfs, column_on):
    """
    Take a bunch of dfs and put them all together given a common column 
    
    Parameters:
        list_of_dfs: List of Pandas DataFrame to merge together
        column_on: String. Column name common to all data frames. 
    
    Returns:
        Merged Pandas Data Frame
    """

    full = reduce(lambda left, right: pd.merge(left, right, how="inner", on=column_on), list_of_dfs)
    
    return full

In [ ]:
personal_df = personal_df_enhancement(personal_df)
country_df = country_df_enhancement(country_df, country_code_df)
career_df = career_df_enhancement(career_df, titles_df)
full = merge_it([personal_df, career_df, country_df], column_on="uuid")

### Analysis

In [30]:
def create_final_df(df, groupby_columns, dropna=False):
    """
    Given a dataframe and a subset of its columns, return a grouped dataframe along with the items count and percentage.

    Paremeters:
        df: Pandas Data Frame coming from the merge_it function.
        groupby_columns: List. Subset of column names that are present in the data frame passed as df.
            First element must be "uuid". Second and so on will be used to group the data frame.
            Second element will be first grouping level, Third element second grouping level, etc. 
            i.e.: ["uuid", "country", "job_title", "rural"]
        dropna: Boolean. Returns a data frame with 0 if there were no matches (default False).
        
    Returns:
        Pandas data frame. I.e:
            |  Country  |  Job Title  |  Gender  |  Quantity  |  Percentage  |
            |  Croatia  | Data Miner  |  Female  |      5     |       1      |
            |  Croatia  | Data Miner  |   Male   |      7     |       2      |
        
    """
    main_df = df[groupby_columns]
    grouped_df = main_df.groupby(by=groupby_columns[1:]).count()
    
    # Computting percentage value and adding it to a new column "percentage".
    grouped_df["percentage"] = grouped_df.groupby(level=0).transform(lambda x : 100 * x/x.sum())
    
    # Turning multi-index dataframe into a single-index one.
    grouped_df.reset_index(inplace=True)
    
    # Filling NaN values with 0
    grouped_df[["uuid", "percentage"]] = grouped_df[["uuid", "percentage"]].fillna(0)
    
    # Rounding values to three decimal points
    grouped_df["uuid"] = grouped_df["uuid"].apply(int)
    grouped_df["percentage"] = grouped_df["percentage"].apply(round, args=(3,))
    
    # Renaming columns
    grouped_df.columns = ["Country", "Job Title", "Gender", "Quantity", "Percentage"]
    
    if dropna:
        grouped_df = grouped_df[grouped_df.Quantity > 0]
        grouped_df.reset_index(drop=True, inplace=True)
    
    return grouped_df

In [31]:
final_gender = create_final_df(full, ["uuid", "country", "job_title", "gender"], dropna=True)
final_rural = create_final_df(full, ["uuid", "country", "job_title", "rural"], dropna=True)
final_age = create_final_df(full, ["uuid", "country", "job_title", "age_group"], dropna=True)

### Reporting

In [18]:
def save_it(df, path, sep=","):
    """
    Save a pandas data frame as a csv file.
    
    Parameters:
        df: Pandas DataFrame.
        path: String. Filepath. I.e.: "../data/final_gender.csv"
        sep: String. File delimiter, comma set as default.
        
    Returns:
        None. Saves df as csv file. 
    """
    
    df.to_csv(path, index=False)
    print("DataFrame saved to:", path)
    return None

In [34]:
save_it(final_gender, "../data/processed/final_gender.csv")
save_it(final_rural, "../data/processed/final_rural.csv")
save_it(final_age, "../data/processed/final_age.csv")

DataFrame saved to: ../data/processed/final_gender.csv
DataFrame saved to: ../data/processed/final_rural.csv
DataFrame saved to: ../data/processed/final_age.csv
